<a href="https://colab.research.google.com/github/prchlmrie/Covid-Sales-Analysis/blob/main/Covid_Sales_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import Necessary libraries
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy import stats
from scipy import stats


#Seaborn Settings
sns.set_style("whitegrid")
sns.set_context("poster")
sns.set(color_codes=True)

%matplotlib inline
warnings.simplefilter('ignore')


## Dataset Column information

- **Type**:  Physical or online store
- **Date**:	Daily dates
- **Product Name**:	Product name
- **Retail Price - Member:**	SRP discounted price for members **(Candidate Target#5)**
- **Retail Price - Non Member:**	SRP increase in retail price **(Candidate Target#6)**
- **Unit Price**:	How much did the retailer get the product for(Additional Insight)
- **Color State**:	Categorical Variables that state color state (Fresh, Pristine, Discolored)
- **Shelf Life**:	Shelf life in days of product(1-3 days)
- **Volume sold Members**:	Number of units sold to members
- **Volume sold to Non Members**:	Number of units sold to non members
- **Total Volume Sold**:	Sales Volume per day of a particular product
- **Competitor price**:	Competitor price
- **Lockdowns**:	One-hot encoded variable on whether a lockdown is present at that date or not.
- **Precipitation**:	Chance of rain at particular day.
- **Average temp**:	Average temperature that day.
- **Covid Cases**:	Number of covid cases at that area
- **Day of week**: A column indicating the day of week
- **Weekday_Weekend**: A column indicating if the day is a workday or weekend
- **Delivered_FROM_LA_MEM:**	Number of units delivered to loyalty membership customers residing in LA **(Candidate Target#1)**
- **Delivered_OUT_LA_MEM:**	Number of units delivered to loyalty membership customers residing in LA **(Candidate Target#2)**
- **Delivered_FROM_LA_NON_MEM:**	Number of units delivered to loyalty membership customers residing in LA **(Candidate Target#3)**
- **Delivered_OUT_LA_NON_MEM:**	Number of units delivered to loyalty membership customers residing in LA **(Candidate Target#4)**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Load datasets
df = pd.read_excel('/content/drive/MyDrive/Colab notebooks/Lab - Analytics/Master Dataset_mod.xlsx', sheet_name=1, parse_dates=True) #Importing source dataset

In [ ]:
#Check first 5 records
df.head()

,Type,Date,Product Name,Retail Price - Member,Retail Price - Non Member,Unit Price,Shelf Life,Members Volume,Non-Members Volume,Total Volume Sold,Lockdowns,PRCP,TAVG,Covid Cases,Delivered_FROM_LA_MEM,Delivered_OUT_LA_MEM,Delivered_FROM_LA_NON_MEM,Delivered_OUT_LA_NON_MEM,Day of Week,Weekday_Weekend
0,Online,2020-01-01,1lb Field Grown Tomatoes,2.297069,2.332050,1.632435,1,210,14,224,0,0.00,38,0,168,42,13,1,Wednesday,Weekday
1,Online,2020-01-02,1lb Field Grown Tomatoes,2.251128,2.285409,1.599786,2,118,64,182,0,0.42,35,0,89,29,58,6,Thursday,Weekday
2,Online,2020-01-03,1lb Field Grown Tomatoes,2.182216,2.215448,1.550813,3,129,109,238,0,0.12,34,0,121,8,107,2,Friday,Weekday
3,Online,2020-01-04,1lb Field Grown Tomatoes,2.187685,2.221000,1.554700,1,70,11,81,0,0.00,40,0,62,8,10,1,Saturday,Weekend
4,Online,2020-01-05,1lb Field Grown Tomatoes,2.143931,2.176580,1.523606,2,42,28,70,0,0.00,45,0,35,7,23,5,Sunday,Weekend


In [ ]:
#Check last 5 records
df.tail()

,Type,Date,Product Name,Retail Price - Member,Retail Price - Non Member,Unit Price,Shelf Life,Members Volume,Non-Members Volume,Total Volume Sold,Lockdowns,PRCP,TAVG,Covid Cases,Delivered_FROM_LA_MEM,Delivered_OUT_LA_MEM,Delivered_FROM_LA_NON_MEM,Delivered_OUT_LA_NON_MEM,Day of Week,Weekday_Weekend
1822,Online,2021-08-27,Cavendish Bananas 1lb,0.575319,0.58408,0.408856,2,51,25,76,0,0.0,77,4341233,42,9,22,3,Friday,Weekday
1823,Online,2021-08-28,Cavendish Bananas 1lb,0.557707,0.56620,0.396340,3,49,28,77,0,0.0,85,4355018,35,14,25,3,Saturday,Weekend
1824,Online,2021-08-29,Cavendish Bananas 1lb,0.587060,0.59600,0.417200,1,70,7,77,0,0.0,84,4365018,58,12,6,1,Sunday,Weekend
1825,Online,2021-08-30,Cavendish Bananas 1lb,0.575319,0.58408,0.408856,2,46,24,70,0,0.0,77,4375018,37,9,21,3,Monday,Weekday
1826,Online,2021-08-31,Cavendish Bananas 1lb,0.557707,0.56620,0.396340,3,52,7,59,0,0.0,71,4385018,42,10,7,0,Tuesday,Weekday


In [ ]:
#Check the columns and datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1827 entries, 0 to 1826
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Type                       1827 non-null   object        
 1   Date                       1827 non-null   datetime64[ns]
 2   Product Name               1827 non-null   object        
 3   Retail Price - Member      1827 non-null   float64       
 4   Retail Price - Non Member  1827 non-null   float64       
 5   Unit Price                 1827 non-null   float64       
 6   Shelf Life                 1827 non-null   int64         
 7   Members Volume             1827 non-null   int64         
 8   Non-Members Volume         1827 non-null   int64         
 9   Total Volume Sold          1827 non-null   int64         
 10  Lockdowns                  1827 non-null   int64         
 11  PRCP                       1827 non-null   float64       
 12  TAVG  

## EDA Workflow

- Check null columns, data sparsity.
- Display summary statistics
- Proceed to feature engineering.

### Initial Data Checks & Summary statistics

## Summary Statistics for Non Membership and Membership subscribed customers

- Members drive sales volume more than non members
- In general, sales volume generated by members is more than half of what non members contribute.

In [ ]:
#Get the number of null values per column
df.isnull().sum()

,0
Type,0
Date,0
Product Name,0
Retail Price - Member,0
Retail Price - Non Member,0
Unit Price,0
Shelf Life,0
Members Volume,0
Non-Members Volume,0
Total Volume Sold,0


In [ ]:
#Describe non-numerical columns
df.describe(exclude=[np.number])

,Type,Date,Product Name,Day of Week,Weekday_Weekend
count,1827,1827,1827,1827,1827
unique,1,NaN,3,7,2
top,Online,NaN,1lb Field Grown Tomatoes,Wednesday,Weekday
freq,1827,NaN,609,261,1305
mean,NaN,2020-10-31 00:00:00,NaN,NaN,NaN
min,NaN,2020-01-01 00:00:00,NaN,NaN,NaN
25%,NaN,2020-06-01 00:00:00,NaN,NaN,NaN
50%,NaN,2020-10-31 00:00:00,NaN,NaN,NaN
75%,NaN,2021-04-01 00:00:00,NaN,NaN,NaN
max,NaN,2021-08-31 00:00:00,NaN,NaN,NaN


### Analysis Part
Answer guide questions below

In [ ]:
#Question #1: Get the sum of "Total volume sold of product: 1lb Field Grown Tomatoes"
df[(df.Type == "Online") & (df['Product Name'] == '1lb Field Grown Tomatoes')]['Total Volume Sold'].sum()

74217

In [ ]:
#Question #2: What are the sum and average sales per day of the previous product?
df[(df['Product Name'] == '1lb Field Grown Tomatoes')][['Day of Week', 'Total Volume Sold']].groupby(['Day of Week']).agg({'Total Volume Sold': [sum, np.mean]})

Total Volume Sold            
                          sum        mean
Day of Week                              
Friday                  10771  123.804598
Monday                  10619  122.057471
Saturday                10937  125.712644
Sunday                  10462  120.252874
Thursday                10529  121.022989
Tuesday                 10375  119.252874
Wednesday               10524  120.965517

In [ ]:
#Question 3: Print the total revenue of the product before for members
df['revenue_mem'] = df[(df['Product Name'] == '1lb Field Grown Tomatoes')]['Retail Price - Member'] * df[(df['Product Name'] == '1lb Field Grown Tomatoes')]['Members Volume']
print(df['revenue_mem'].sum())

99611.66593808499


In [ ]:
#Question 4: Print the total revenue of the product before for non-members?
df['revenue_nonmem'] = df[(df['Product Name'] == '1lb Field Grown Tomatoes')]['Retail Price - Non Member'] * df[(df['Product Name'] == '1lb Field Grown Tomatoes')]['Retail Price - Non Member']
print(df['revenue_nonmem'].sum())

2155.4602574236796
